In [1]:
import time
import pandas as pd
import requests
import json
import numpy as np

from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
url = 'https://www.premierleague.com/tables'
year_table = "2018/19"

tables = []

# Select Wait Time and Season
wait_time = 1
start_year = list(range(1995, 2020 + 1))

browser = webdriver.Chrome()
browser.maximize_window()
wait = WebDriverWait(browser, 60)


browser.get(url)

# Accept Cookies
xpath_cookies = '//div[@class="btn-primary cookies-notice-accept"]/span'
cookies = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_cookies)))
time.sleep(wait_time)
cookies.click()

# Close No to Racism Tab
xpath_racism = '//a[@id="advertClose"]'
racism = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_racism)))
time.sleep(wait_time)
racism.click()


for year in start_year:
    season_year = str(year)
    abbr_year = str(year + 1)
    season_year = season_year + "/" + abbr_year[-2:]

    # Open Season Tab
    xpath_season = '//div[@data-dropdown-current="compSeasons"]'
    season = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_season)))
    time.sleep(wait_time)
    season.click()

    # Select Season
    xpath_year = f'//li[@data-option-name="{season_year}"]'
    year = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_year)))
    time.sleep(wait_time)
    year.click()

    sleep(wait_time)
    current_url = browser.current_url
    #print(current_url)
    current_url = current_url.split("=")
    current_url = current_url[2]
    idx = current_url.find("&")
    current_url = current_url[:idx]
    current_url
    
    headers = {
    'authority': 'footballapi.pulselive.com',
    'sec-ch-ua': '^\\^',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'accept': '*/*',
    'origin': 'https://www.premierleague.com',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.premierleague.com/',
    'accept-language': 'en-US,en;q=0.9',
    }

    params = (
        ('compSeasons', current_url),
        ('altIds', 'true'),
        ('detail', '2'),
        ('FOOTBALL_COMPETITION', '1'),
    )

    response = requests.get('https://footballapi.pulselive.com/football/standings', headers=headers, params=params)
    info = json.loads(response.text)  
    
    club_id = []
    name_club = []
    position_club = []

    for i in range(20): 
    # id and name
        c_id = info["tables"][0]["entries"][i]['team']['id']
        club_id.append(c_id)
        name = info["tables"][0]["entries"][i]['team']['shortName']
        name_club.append(name)
    # position
        position = info["tables"][0]["entries"][i]["position"]
        position_club.append(position)

    d = {"Year": season_year,
     "Position": position_club,
     "ID": club_id,
     "Name": name_club,
        }
    df = pd.DataFrame(data=d)
    df = df.pivot(index=["Position"], columns=["Year"])
    #display(df)
    tables.append(df)
    
browser.close()
browser.quit()

final_table = pd.DataFrame()

for i in range(len(tables)):
    final_table = pd.concat([final_table, tables[i]], axis=1)
    
display(final_table)

,ID,Name,ID,Name,ID,Name,ID,Name,ID,Name,...,ID,Name,ID,Name,ID,Name,ID,Name,ID,Name
Year,1995/96,1995/96,1996/97,1996/97,1997/98,1997/98,1998/99,1998/99,1999/00,1999/00,...,2016/17,2016/17,2017/18,2017/18,2018/19,2018/19,2019/20,2019/20,2020/21,2020/21
Position,,,,,,,,,,,,,,,,,,,,,
1,12,Man Utd,12,Man Utd,1,Arsenal,12,Man Utd,12,Man Utd,...,4,Chelsea,11,Man City,11,Man City,10,Liverpool,11,Man City
2,23,Newcastle,23,Newcastle,12,Man Utd,1,Arsenal,1,Arsenal,...,21,Spurs,12,Man Utd,10,Liverpool,11,Man City,12,Man Utd
3,10,Liverpool,1,Arsenal,10,Liverpool,4,Chelsea,9,Leeds,...,11,Man City,21,Spurs,4,Chelsea,12,Man Utd,26,Leicester
4,2,Aston Villa,10,Liverpool,4,Chelsea,9,Leeds,10,Liverpool,...,10,Liverpool,10,Liverpool,21,Spurs,4,Chelsea,4,Chelsea
5,1,Arsenal,2,Aston Villa,9,Leeds,25,West Ham,4,Chelsea,...,1,Arsenal,4,Chelsea,1,Arsenal,26,Leicester,25,West Ham
6,7,Everton,4,Chelsea,3,Blackburn,2,Aston Villa,2,Aston Villa,...,12,Man Utd,1,Arsenal,12,Man Utd,21,Spurs,21,Spurs
7,3,Blackburn,19,Sheffield Wed,2,Aston Villa,10,Liverpool,29,Sunderland,...,7,Everton,43,Burnley,38,Wolves,38,Wolves,10,Liverpool
8,21,Spurs,22,Wimbledon,25,West Ham,28,Derby,26,Leicester,...,20,Southampton,7,Everton,7,Everton,1,Arsenal,7,Everton
